# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
# loading messages dataset
messages = pd.read_csv('messages.csv')
messages.head(20)

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct


In [3]:
# Some checks about messages dataset
messages.shape

(26248, 4)

In [4]:
# About the genre column data
messages.genre.value_counts()

news      13068
direct    10782
social     2398
Name: genre, dtype: int64

In [5]:
# what's a message?
messages.iloc[13188].message

'She added: "We are not going to treat the Zimbabwean government as if it was an illegitimate government or President (Robert) Mugabe as if he was an illegitimate president."'

In [6]:
# loading categories dataset
categories = pd.read_csv('categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [7]:
# How about categories shape?
categories.shape

(26248, 2)

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [8]:
# merging both datasets
df = messages.merge(categories, on='id', how='left')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [9]:
# The new shape
df.shape

# Well it change the number of rows, how is that?

(26386, 5)

In [10]:
# Let's see for unique id numbers
np.sum(np.unique(df.id, return_counts=True)[1])

26386

In [11]:
# Don't get it, let me check the info, and clean it later.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26386 entries, 0 to 26385
Data columns (total 5 columns):
id            26386 non-null int64
message       26386 non-null object
original      10246 non-null object
genre         26386 non-null object
categories    26386 non-null object
dtypes: int64(1), object(4)
memory usage: 1.2+ MB


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [12]:
# create a dataframe of the 36 individual category columns
categories = pd.DataFrame(columns=['' for x in range(36)])
categories.head()

,,,,,,,,,,,...,,,,,,,,,,


In [13]:
# select the first row of the categories dataframe
row = df.loc[0, 'categories']

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = [x[:-2] for x in row.split(';')]
print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [14]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [15]:
for number, column in enumerate(categories):
    # set each value to be the last character of the string
    categories[column] = df.categories.astype(str).apply(lambda x: x.split(';')[number][-1])
    print(number,column)
    # convert column from string to numeric
    categories[column] = categories[column].apply(int)
categories.head()

0 related
1 request
2 offer
3 aid_related
4 medical_help
5 medical_products
6 search_and_rescue
7 security
8 military
9 child_alone
10 water
11 food
12 shelter
13 clothing
14 money
15 missing_people
16 refugees
17 death
18 other_aid
19 infrastructure_related
20 transport
21 buildings
22 electricity
23 tools
24 hospitals
25 shops
26 aid_centers
27 other_infrastructure
28 weather_related
29 floods
30 storm
31 fire
32 earthquake
33 cold
34 other_weather
35 direct_report


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Let's do a numerical check
# for noise in some data ('related' column this time)
categories.describe()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26386.000000,26386.000000,26386.000000,26386.000000,26386.00000,26386.000000,26386.000000,26386.00000,26386.000000,26386.0,...,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000
mean,0.775032,0.171038,0.004586,0.415144,0.07955,0.049989,0.027477,0.01785,0.032707,0.0,...,0.011711,0.043773,0.278292,0.082506,0.093383,0.010687,0.093269,0.020200,0.052263,0.193777
std,0.435692,0.376549,0.067564,0.492756,0.27060,0.217926,0.163471,0.13241,0.177871,0.0,...,0.107583,0.204594,0.448166,0.275139,0.290974,0.102828,0.290815,0.140687,0.222560,0.395264
min,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
# To fix the numerical problems, let's garantee just 0 and 1 in the data.
categories = (categories>0).astype(int)

In [18]:
# Duplicate the 'related' column to avoid eliminate those rows with 0 values
# the duplicated new column will have '1' where 'related' is '0'
# that warantees some issues fixed in the ML step because we are avoiding
# rows full of '0's.
categories['not_related'] = categories['related'].apply(lambda x: [1,0][x])

In [19]:
# For ML training purposes, let's erase the columns with '0' values, this
# could bring issues in some ML algorithms.
for col in categories.columns:
    if categories[col].sum() == 0:
        categories.drop(col, axis=1, inplace=True)

In [20]:
# What we have at the moment
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
categories.tail()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,not_related
26381,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
26382,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
26383,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26384,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [21]:
# drop the original categories column from `df`

df.drop('categories', axis=1, inplace=True)
df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that could pass over Haiti,Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ak timoun yo. Mesi se john jean depi Monben kwochi.",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.,UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.,direct
4,12,"says: west side of Haiti, rest of the country today and tonight",facade ouest d Haiti et le reste du pays aujourd hui et ce soir,direct


In [22]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories], axis = 1)
df.tail()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,not_related
26381,30261,"The training demonstrated how to enhance micronutrient, protein and energy intake for meals using locally grown horticulture produce, rice and fish.",NaN,news,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
26382,30262,A suitable candidate has been selected and OCHA Jakarta is currently working on her three month-contract starting on the 01st July 2005.,NaN,news,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
26383,30263,"Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",NaN,news,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26384,30264,"Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.",NaN,news,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26385,30265,"A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.",NaN,news,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [23]:
# check number of duplicates
df.duplicated('id').value_counts()

False    26180
True     206  
dtype: int64

In [24]:
# drop duplicates
df_dropped = df.drop_duplicates()

In [25]:
# check number of duplicates
df_dropped.duplicated('id').value_counts()

False    26180
True     35   
dtype: int64

In [26]:
# Let's understand what is happening with the duplicated data
df_dropped[df_dropped.duplicated('id', keep=False)]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,not_related
162,202,?? port au prince ?? and food. they need goverment aid and international aid thak you. god bless haiti,"p bay pap la syen ak manje. Yo bezwen ed gouvenman an ak d entenasyonal. Mesi, BonDye beni Ayiti.",direct,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
163,202,?? port au prince ?? and food. they need goverment aid and international aid thak you. god bless haiti,"p bay pap la syen ak manje. Yo bezwen ed gouvenman an ak d entenasyonal. Mesi, BonDye beni Ayiti.",direct,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
713,862,What is the address of the radio station? I ask because I need to drop off a file for the committee we created. Thanks in advance!,Ki adres radyo a? Paske m bezwen al depoze dosye ki bay detay sou komite nou fmea. Mesi davans!,direct,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
714,862,What is the address of the radio station? I ask because I need to drop off a file for the committee we created. Thanks in advance!,Ki adres radyo a? Paske m bezwen al depoze dosye ki bay detay sou komite nou fmea. Mesi davans!,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1413,1652,"please we need water, food and tents, we have 101 chlidren. .. please come rescue","p jwen dlo, manje, tant pou nou demi nou gen 101 timoun vinn sove n",direct,1,1,0,1,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1414,1652,"please we need water, food and tents, we have 101 chlidren. .. please come rescue","p jwen dlo, manje, tant pou nou demi nou gen 101 timoun vinn sove n",direct,1,1,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2836,3250,I thought you gave my number and this was serious. Meanwhile today has been 15 days and we still haven't gotten anything,Mwen tepanse le nou tebay nimewo a se yon afserye padan setan jodi a f 15 jou nou paka jwenn anwyen,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2837,3250,I thought you gave my number and this was serious. Meanwhile today has been 15 days and we still haven't gotten anything,Mwen tepanse le nou tebay nimewo a se yon afserye padan setan jodi a f 15 jou nou paka jwenn anwyen,direct,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3387,3882,"I am in Carefour Feuilles, Dkayet neighborhood. We need potable water because diarrhea is rampant here and is undermining our health.",Mwen nan kafou fy zon dekayt nou beswen dlo potab paske dyare ap fini ave nou.,direct,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3388,3882,"I am in Carefour Feuilles, Dkayet neighborhood. We need potable water because diarrhea is rampant here and is undermining our health.",Mwen nan kafou fy zon dekayt nou beswen dlo potab paske dyare ap fini ave nou.,direct,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [27]:
# They are pretty similar, Let's drop the rows with the same 'id'
# Keeping the last one.
df_dropped = df_dropped.drop_duplicates('id', keep='last')

In [28]:
# Showing results
df_dropped.duplicated('id').value_counts()

False    26180
dtype: int64

In [29]:
# Final dataset
df_dropped.shape

(26180, 40)

In [30]:
# Numerical check of the final dataset
df_dropped.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,not_related
count,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.00000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000
mean,15227.563942,0.766272,0.170474,0.004507,0.414095,0.079488,0.050115,0.027617,0.017991,0.032773,0.063751,0.111421,0.088197,0.015432,0.023033,0.011383,0.033384,0.045531,0.13136,0.064973,0.045760,0.050764,0.020321,0.006073,0.010810,0.004584,0.011803,0.043812,0.278189,0.082047,0.093201,0.010772,0.093659,0.020206,0.052445,0.193506,0.233728
std,8827.269301,0.423209,0.376056,0.066986,0.492574,0.270504,0.218186,0.163875,0.132920,0.178046,0.244313,0.314659,0.283587,0.123264,0.150011,0.106083,0.179641,0.208470,0.33780,0.246483,0.208968,0.219519,0.141098,0.077696,0.103409,0.067549,0.108000,0.204681,0.448116,0.274442,0.290719,0.103228,0.291360,0.140708,0.222926,0.395054,0.423209
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7449.750000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15665.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22927.250000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [31]:
# More checks of the final dataset
df_dropped.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,not_related
0,2,Weather update - a cold front from Cuba that could pass over Haiti,Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ak timoun yo. Mesi se john jean depi Monben kwochi.",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.,UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country today and tonight",facade ouest d Haiti et le reste du pays aujourd hui et ce soir,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 6.1. Doing something with imbalances.
- Checking first the how imbalanced is the data.
- Oversampling the small data.
- Undersampling the biggest one.

In [32]:
# Let's check how imbalanced is the data in relation to
# 'related' column data
100*df_dropped[categories.columns].sum()/len(df)

related                   76.028955
request                   16.914273
offer                     0.447207 
aid_related               41.086182
medical_help              7.886758 
medical_products          4.972334 
search_and_rescue         2.740089 
security                  1.785038 
military                  3.251724 
water                     6.325324 
food                      11.055105
shelter                   8.750853 
clothing                  1.531115 
money                     2.285303 
missing_people            1.129387 
refugees                  3.312363 
death                     4.517547 
other_aid                 13.033427
infrastructure_related    6.446600 
transport                 4.540287 
buildings                 5.036762 
electricity               2.016221 
tools                     0.602592 
hospitals                 1.072538 
shops                     0.454787 
aid_centers               1.171076 
other_infrastructure      4.347002 
weather_related           27

In [33]:
# Checking how imbalaced the data is
df_dropped[categories.columns].sum()

related                   20061
request                   4463 
offer                     118  
aid_related               10841
medical_help              2081 
medical_products          1312 
search_and_rescue         723  
security                  471  
military                  858  
water                     1669 
food                      2917 
shelter                   2309 
clothing                  404  
money                     603  
missing_people            298  
refugees                  874  
death                     1192 
other_aid                 3439 
infrastructure_related    1701 
transport                 1198 
buildings                 1329 
electricity               532  
tools                     159  
hospitals                 283  
shops                     120  
aid_centers               309  
other_infrastructure      1147 
weather_related           7283 
floods                    2148 
storm                     2440 
fire                      282  
earthqua

In [34]:
# Checking the proportion of rows with only one feature value
# To analyze if we will clean these rows or not
db = df_dropped[categories.columns]
for col in categories.columns:
    d_base = db[db[col] == 1]
    print(col, (d_base.sum(axis=1)==1).sum())

related 5300
request 0
offer 0
aid_related 0
medical_help 0
medical_products 0
search_and_rescue 0
security 0
military 0
water 0
food 0
shelter 0
clothing 0
money 0
missing_people 0
refugees 0
death 0
other_aid 0
infrastructure_related 0
transport 0
buildings 0
electricity 0
tools 0
hospitals 0
shops 0
aid_centers 0
other_infrastructure 0
weather_related 0
floods 0
storm 0
fire 0
earthquake 0
cold 0
other_weather 0
direct_report 0
not_related 6119


In [35]:
# An approach to fix a little bit the imbalanced data, doing Oversampling
# for a group of features with less data
# Just a bit, too much oversampling push our model to overfit
def Oversampling_Imbalanced_Data(data, target_col, factor=2, top_data=0.05, replace=True):
    '''
    This function return the new dataset taking a sample of the existing data.
    
    Function parameters:
    
        Required:
            
            data : The original dataset to be oversampled.
            
            target_col : list ; 
                the columns of the featured data or categories to be checked.
            
        Optional:
            
            factor : float; default = 2 (2x), example: 0.0 (0x) - 10.0 (10x)
                the factor that multiplies the number of rows to be repeated.
                
            top_data : float; default = 0.05 (5%), example: 0.0 (0%) - 0.1 (10%)
                the percentage of data represented that will include the features to 
                be repeated.
                
            replace : True or False;
                This determine if wants to repeat some of the data already sampled. 
                If you don't want to repeat data more than once then the factor can't
                be more than 1.
                
        Return:
        
            new_data : the new dataset oversampled.
        
    '''

    new_data = data.copy()
    # Let's walk through every feature or category
    for col in target_col:
        # taking a sub dataframe where all rows with this column is '1'
        d_base = data[data[col] == 1]
        # If there is data and the number of rows is below the top_data value:
        if (len(d_base) > 0) and ((len(d_base)/len(data))<top_data):
            # Let's take the samples and add them to the new dataframe
            d_samples = d_base.sample(n = int(factor*len(d_base)), replace=replace)
            new_data = pd.concat([new_data, d_samples], ignore_index=True)
            
    # Returning the new oversampled dataset.
    return new_data

In [36]:
# Now lets define a function to make undersampling to the biggest
# features or categories represented.
# In this case we are only taking the related and not_related columns,
# deleting only the rows where there is only a '1' in their features, and
# '0' in the rest of the categories.

def Undersampling_Imbalanced_Related_Data(data, target_col, factor=0.5):
    '''
    This function return the new dataset removing a sample of the existing data
    of features than only have values in their columns alone, and '0' in the others.
    
    Function parameters:
    
        Required:
            
            data : The original dataset to be undersampled.
            
            target_col : list ;
                the columns of the featured data or categories to be checked.
            
        Optional:
            
            factor : float; default = 0.5 (50%), example: 0.0 (0%) - 1.0 (100%)
                the factor that represent the percentage been elminated from the
                existing data.
                
        Return:
        
            new_data : the new dataset undersampled.
        
    '''

    new_data = data.copy()
    # calculating the number of rows to delete in the rows that only have one value.
    # In this case they are only the related and not_related columns.
    l = int(len(new_data[new_data[target_col].sum(axis=1) == 1]) * factor)
    # Taking a sample of that data.
    d_samples = new_data[new_data[target_col].sum(axis=1) == 1].sample(n = l, replace=False)
    # Finally we create new_data dataset from the data without these rows, and the
    # sample selected.
    new_data = pd.concat([new_data[new_data[target_col].sum(axis=1) > 1],d_samples], ignore_index=True)
    
    return new_data

In [37]:
df_dropped.shape

(26180, 40)

In [38]:
# Let's do oversampling of the data that is below the 3,5% of the representation,
# and just add 70% more of that data without repeating.
dfx = Oversampling_Imbalanced_Data(df_dropped, categories.columns, factor=0.7, top_data=0.035, replace=False)

In [39]:
dfx.shape

(30768, 40)

In [40]:
# And finally some undersampling of the related features.
# Let's reduce it 60%
dfx = Undersampling_Imbalanced_Related_Data(dfx, categories.columns, 0.4)

In [41]:
dfx.shape

(23916, 40)

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [44]:
# Saving the dataset
engine = create_engine('sqlite:///DisasterResponse.db')
dfx.to_sql('DisasterResponse', engine, index=False, if_exists='replace')

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.